# Dewan Lab EPM Analysis

## STEP 1: Always Execute! Load Libraries and User Settings
### STEP 1A: Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

import cv2
import numpy as np
import pandas as pd
from scipy import ndimage

from pathlib import Path
from tqdm import tqdm, trange

from dewan_calcium import plotting, deconv
from dewan_calcium.helpers import IO, parse_json, EPM
from dewan_calcium.helpers.project_folder import ProjectFolder

print("Importing required packages complete!")

### STEP 1B: User Configurables

In [ ]:
animal = 'ANIMAL_GOES_HERE'
date = 'DATE_GOES_HERE'

EXPERIMENT_TIME = 10
LED_CUTOFF = 0.95
ENDOSCOPE_FRAMERATE = 10  # Hz
PSEUDOTRIAL_LEN_S = 2

# Endoscope Framerate = 10Hz
# Rise time = 80ms
# Decay time = 400ms

peak_args = {
    'decay': 400, #ms 
    'rise': 80,  #ms
    'ISI': 100, # inter-spike-interval; ms
    'height': 1 # 1 SD
}

In [ ]:
# Create Project Folder to Gather and Hold all the File Paths

#test_path = Path("/mnt/dev/Test_Data/Odor/VGLUT-20")  # On Fedora
#test_path = Path("C:\\Projects\\Test_Data\\EPM\\VGLUT-20")  # On Desktop
#test_path = Path('D:/Test_Data/EPM/VGLUT-20') # Laptop Windows
test_path = Path('D:\\Projects\\Test_Data\\EPM\\VGLUT-20') # C474B

project_folder = ProjectFolder('EPM', project_dir=test_path)
file_header = animal + '-' + date + '-'

In [ ]:
# If this is the first time the project folder has been created,
# move the files to the appropriate directories and then run this cell, otherwise skip this cel
project_folder.get_data()

In [ ]:
# Get settings from imaging session and display them for the user

gain, LED_power, focal_planes = parse_json.get_session_settings(project_folder.raw_data_dir.session_json_path)

print(f'Recording Gain: {gain}')
print(f'LED Power: {LED_power}')
print(f'Focal Plane(s): {focal_planes}')

## 2A: Import and pre-process the raw data

In [ ]:
#STEP 2A.1: LOAD DLC DATA

tracked_points = pd.read_hdf(project_folder.raw_data_dir.points_h5_path)  # Load tracked points
labeled_video = cv2.VideoCapture(str(project_folder.raw_data_dir.labeled_video_path))  # Load Video
VIDEO_FPS = labeled_video.get(cv2.CAP_PROP_FPS)

In [ ]:
#STEP 2A.2: LOAD INSCOPIX DATA

cell_trace_data = pd.read_csv(project_folder.inscopix_dir.cell_trace_path, engine='pyarrow')
GPIO_data = pd.read_csv(project_folder.inscopix_dir.GPIO_path, header=0, engine='pyarrow')
all_cell_props = pd.read_csv(project_folder.inscopix_dir.props_path, header=0, engine='pyarrow')
cell_outlines = parse_json.get_outline_coordinates(project_folder.inscopix_dir.contours_path)

In [ ]:
# STEP 2A.2: PREPROCESSING

# STEP 2A.2.1: Drop the first row which contains all 'undecided' labels which is the Inscopix default label.
cell_trace_data = cell_trace_data.drop([0])

# STEP 2A.2.2: Force all dF/F values to be numbers and round times to 2 decimal places
cell_trace_data = cell_trace_data.apply(pd.to_numeric, errors='coerce')


# Set the times as the index so the listed data is all dF/F values
cell_trace_data[cell_trace_data.columns[0]] = cell_trace_data[cell_trace_data.columns[0]].round(2)
cell_trace_data = cell_trace_data.set_index(cell_trace_data.columns[0]) 

# STEP 2A.2.3: Remove spaces from column names and contents
cell_trace_data.columns = cell_trace_data.columns.str.replace(" ", "")
GPIO_data.columns = GPIO_data.columns.str.replace(" ", "")
GPIO_data['ChannelName'] = GPIO_data['ChannelName'].str.replace(" ", "")

# STEP 2A.2.4: Reduce properties to only include the cells with only one component
all_cell_props = all_cell_props[all_cell_props['NumComponents']==1]  # We only want cells that have one component
all_cell_props = all_cell_props.drop(columns='Status').reset_index(drop=True)
cell_names = all_cell_props['Name'].values

# STEP 2A.2.5: PARSE GPIO DATA
sniff_data = GPIO_data[GPIO_data['ChannelName'] == "GPIO-1"].reset_index(drop=True)
FV_data = GPIO_data[GPIO_data['ChannelName'] == "GPIO-2"].reset_index(drop=True)

# OPTIONAL UNUSED DATA
# running_data = GPIO_data[GPIO_data['ChannelName'] == "GPIO-3"]  # Running Wheel Data
# lick_data = GPIO_data[GPIO_data['ChannelName'] == "GPIO-4"]  # Lick Data


In [ ]:
# STEP 2A.3: PREPROCESSING DLC Data

cols = ['mouse_x', 'mouse_y', 'mouse_p', 'led_x', 'led_y', 'led_p'] 
# Reset the column names to something sensible
tracked_points.columns = cols 

### STEP 2B: Manual Curation

In [ ]:
from dewan_manual_curation import dewan_manual_curation
# STEP 2B.2: Run ManualCuration GUI
curated_cells = dewan_manual_curation.launch_gui(project_folder_override=project_folder, cell_trace_data_override=cell_trace_data, cell_props_override=all_cell_props, cell_contours_override=cell_outlines)
if curated_cells is None:
    print('Error, no good cells selected!')

### STEP 2C: Apply Manual Curation Results and Additional Preprocessing

In [ ]:
# STEP 2C.1: Filter all data by the GoodCells identified in ManualCuration
curated_cell_props = all_cell_props[all_cell_props['Name'].isin(curated_cells)].reset_index(drop=True)
curated_trace_data = cell_trace_data[curated_cells]
cell_names = curated_cell_props['Name']

### STEP 2D: Pickle and Save all preprocessed data

In [ ]:
# Pickle the reorganized CellTraceData incase its needed later
# Saves Cell Traces, GPIO, Odor List, Sniff, FV data, Good Cell Properties, Good Cells, and the labeled max projection
# Once these have been saved, they don't need to be re-run on the same data again unless the data itself is changed

folder = project_folder.analysis_dir.preprocess_dir.path

IO.save_data_to_disk(curated_trace_data, 'curated_trace_data', file_header, folder)
IO.save_data_to_disk(GPIO_data, 'GPIO_data', file_header, folder)
IO.save_data_to_disk(FV_data, 'FV_data', file_header, folder)
IO.save_data_to_disk(curated_cell_props, 'curated_cell_props', file_header, folder)
IO.save_data_to_disk(sniff_data, 'sniff_table', file_header, folder)

IO.save_data_to_disk(tracked_points, 'tracked_points', file_header, folder)
IO.save_data_to_disk(VIDEO_FPS, 'VIDEO_FPS', file_header, folder)

In [ ]:
# Opens the saved pickle files.  If the files have already been saved, code can be re-run
# starting from this point

folder = project_folder.analysis_dir.preprocess_dir.path

curated_trace_data = IO.load_data_from_disk('curated_trace_data', file_header, folder)
GPIO_data = IO.load_data_from_disk('GPIO_data', file_header, folder)
FV_data = IO.load_data_from_disk('FV_data', file_header, folder)
curated_cell_props = IO.load_data_from_disk('curated_cell_props', file_header, folder)
sniff_data = IO.load_data_from_disk('sniff_table', file_header, folder)

cell_names = curated_cell_props['Name']  # List of cells, referenced periodically

tracked_points = IO.load_data_from_disk('tracked_points', file_header, folder)
VIDEO_FPS = IO.load_data_from_disk('VIDEO_FPS', file_header, folder)

In [ ]:
# There may be an instance where the model erroneously identified the LED for very short time periods
# find_index_bins returns the LED on and off bins (anywhere led_p > 0.98)
# true_led_bin ensures that the "true" start bin is selected and the "end" bin is not erroneously selected

LED_indexes = tracked_points.index[tracked_points['led_p'] > LED_CUTOFF].values

led_bins = EPM.find_index_bins(LED_indexes)
true_led_bin = EPM.get_true_bin_index(led_bins, len(tracked_points))

led_on = true_led_bin[0] # The first instance where the LED is 'on'
experiment_frames = int(VIDEO_FPS * 60 * EXPERIMENT_TIME)  # FPS * 60 s/min * experiment length in minutes --> number of frames
end_frame = led_on + experiment_frames
good_points = tracked_points.iloc[led_on:end_frame] # Subset the frames from LED_ON -> EXPERIMENT_TIME minutes later
good_points = good_points.reset_index(drop=True) # Reset the index

# Get X, Y coordinates, cast to int, and combine them into tuples
head_x = good_points['mouse_x'].astype(int)
head_y = good_points['mouse_y'].astype(int)
coordinates = [list(item) for item in zip(head_x, head_y)]
coordinates = np.array(coordinates)

In [ ]:
# Run this cell to interpolate large distance jumps in the DLC tracking data
thresh, num_jumps, coordinates = EPM.interpolate_DLC_coordinates(coordinates) # TODO: Empirically determine the jump threshold 
print(f"There were {num_jumps} jumps that required interpolation.")
print(f"The interpolation threshold used was {thresh}")

In [ ]:
# STEP 3A: Parses the final valve data to identify when the final valve is open vs when it is closed based on TTL pulse from Arduino.
# In the EPM experiment, there is no final valve. However, we are using the same sync signal as used in the odor experiments to signal when the LED is triggered

FV_values = FV_data['Value'].astype(float).values # Get FV Values
num_values = len(FV_values)
valve_status = 0
FV_on_indexes = []
FV_off_indexes = []
for i in trange((num_values - 1), desc="Processing: "):
    valve_val_diff = FV_values[i + 1] - FV_values[i]

    if valve_status == 0:    # Start with valve off
        if valve_val_diff > 10000: # If the difference is a very large positive number, the valve opened
            FV_on_indexes.append(i + 1)
            valve_status = 1 # Set valve state to open
    else:
        if valve_val_diff < -10000: # If the difference is a very laarge negative number, the valve closed
            FV_off_indexes.append(i)
            valve_status = 0 # Set valve state to closed

FV_indexes = pd.DataFrame(zip(FV_on_indexes, FV_off_indexes), columns=['On', 'Off'])

In [ ]:
experiment_start_index = FV_indexes['On'][0]
FV_timestamps = FV_data['Time(s)']
trial_start_time = FV_timestamps[experiment_start_index]  # Trial start time in unix time (s)
trial_end_time = trial_start_time + (EXPERIMENT_TIME * 60)  # End time is whatever the duration of the experiment was in minutes

cell_trace_times = curated_trace_data.index.values

cell_trace_on_index = np.where(cell_trace_times <= trial_start_time)[0][-1]
cell_trace_off_index = np.where(cell_trace_times <= trial_end_time)[0][-1]  # We can't overshoot otherwise the coordinate will not match, so we may drop a single frame

trimmed_trace_data = curated_trace_data.iloc[cell_trace_on_index:cell_trace_off_index, :]

trimmed_cell_trace_times = trimmed_trace_data.index.values
shifted_cell_trace_times = np.subtract(trimmed_cell_trace_times, trimmed_cell_trace_times[0])
rounded_cell_trace_times = np.round(shifted_cell_trace_times, 2)

trimmed_trace_data.index = rounded_cell_trace_times

good_points_index = good_points.index.values
good_points_time = np.divide(good_points_index, VIDEO_FPS)
good_points.index = good_points_time

In [ ]:
## Align Cell Traces with the DLC Data
## Since the DLC data is typically recorded at 6X the rate as the neural data, there is typically multiple data points we can choose for the coordinate of a trace
## For simplicity, we will pick the coordinate that exactly matches the time point of the trace
## In the future we can do some averaging or picking the median, etc. 

trace_coordinate_indexes = []
good_points_index = good_points.index.values

for time in tqdm(trimmed_trace_data.index):  # Check each 
    coordinate_index = np.where(good_points.index <= time)[0][-1]
    trace_coordinate_indexes.append(coordinate_index)

trace_coordinates = coordinates[trace_coordinate_indexes]
trace_coordinates = trace_coordinates.tolist()
trimmed_trace_data.insert(0, 'Coordinate_Index', trace_coordinate_indexes)
trimmed_trace_data.insert(0, 'Coordinates', trace_coordinates)

In [ ]:
## Save the paired coordinates - trace data
folder = project_folder.analysis_dir.preprocess_dir.path

IO.save_data_to_disk(trimmed_trace_data, 'trimmed_trace_data', file_header, folder)
IO.save_data_to_disk(true_led_bin, 'true_led_bin', file_header, folder)

In [ ]:
## Load the ROI selection data
folder = project_folder.analysis_dir.preprocess_dir.path
labeled_video = cv2.VideoCapture(str(project_folder.raw_data_dir.labeled_video_path)) 
trimmed_trace_data = IO.load_data_from_disk('trimmed_trace_data', file_header, folder)
true_led_bin = IO.load_data_from_disk('true_led_bin', file_header, folder)
led_on = true_led_bin[0]

In [ ]:
EPM.display_roi_instructions()

In [ ]:
%matplotlib qt  
# Opens the matplotlib window using the QT backend

labeled_video.set(cv2.CAP_PROP_POS_FRAMES, led_on - 1) # Pull the frame that is our actual start
_, background_image = labeled_video.read()

arm_coordinates = EPM.get_arm_rois(background_image)

# Switch back to using inline displays
%matplotlib inline

individual_regions, original_regions = EPM.get_region_polygons(arm_coordinates)  
# ([open_arm_1, open_arm_2, closed_arm_1, closed_arm_2, center_polygon], [open_arm, closed_arm, center])

fig, ax = plotting.plot_epm_roi(original_regions, background_image)  # TODO: Ensure that the order of arms is still o1, o2, c1, c2 if the EPM is rotated 90 degrees but labeled correctly

In [ ]:
## Save the ROIs and image
folder = project_folder.analysis_dir.figures_dir.subdir('EPM_ROI')

image_path = folder.joinpath('EPM_ROI.pdf')
fig.savefig(str(image_path), dpi=600)

folder = project_folder.analysis_dir.preprocess_dir.subdir('EPM_ROI')
IO.save_data_to_disk(arm_coordinates, 'arm_coordinates', file_header, folder)
IO.save_data_to_disk(individual_regions, 'individual_regions', file_header, folder)
IO.save_data_to_disk(original_regions, 'original_regions', file_header, folder)
IO.save_data_to_disk(background_image, 'background_image', file_header, folder)

In [ ]:
## Load the ROIs and image
folder = project_folder.analysis_dir.preprocess_dir.subdir('EPM_ROI')
arm_coordinates = IO.load_data_from_disk('arm_coordinates', file_header, folder)
individual_regions = IO.load_data_from_disk('individual_regions', file_header, folder)
original_regions = IO.load_data_from_disk('original_regions', file_header, folder)
background_image = IO.load_data_from_disk('background_image', file_header, folder)

folder = project_folder.analysis_dir.preprocess_dir.path
trimmed_trace_data = IO.load_data_from_disk('trimmed_trace_data', file_header, folder)
curated_cell_props = IO.load_data_from_disk('curated_cell_props', file_header, folder)
cell_names = curated_cell_props['Name']

In [ ]:
# Get region for each time point and distance from occupied region
animal_coordinates = trimmed_trace_data['Coordinates']
coordinate_locations, region_indexes = EPM.get_regions(animal_coordinates, individual_regions) # What region each (x, y) is in

In [ ]:
# Run this cell if you would like to interpolate any position that fell outside the 5 regions
void_indexes = np.where(coordinate_locations == "The_Void")[0]  # "The_Void" is the location assigned to points that fall outside of the five regions (open1, open2, closed1, closed2, center). This can occur if the animal looks over the side of the open arms
void_index_bins = EPM.find_index_bins(void_indexes)
coordinate_locations, region_indexes = EPM.replace_the_void(coordinate_locations, region_indexes, void_index_bins)

In [ ]:
# Optional: Run this cell if you would like to calculate the "distance" into the arm the animal has traveled
coordinate_pairs = list(zip(animal_coordinates, region_indexes))
distances = EPM.get_distances(individual_regions, coordinate_pairs)  # Return distance from animal -> occupied region
# distances = EPM.normalize_distance(individual_regions, coordinate_locations, distances)
# If desired, the distances can be normalized to the length of the arms. All positions become 'percentages' of the length along the arm
trimmed_trace_data.insert(0, 'Distance', distances)

In [ ]:
# Add animal Location to the main list of trace data
trimmed_trace_data.insert(0, 'Location', coordinate_locations)

In [ ]:
## PSEUDOTRIAL COLLECTION
# Find transitions from region -> region
# Gather all visits per each region
# Calculate some statistics about our 'pseudotrials'

animal_locations = trimmed_trace_data['Location']
transitions, arm_indexes = EPM.find_region_transitions(animal_locations)

In [ ]:

pseudotrials, trial_stats = EPM.get_pseudotrials(arm_indexes, transitions, PSEUDOTRIAL_LEN_S, ENDOSCOPE_FRAMERATE)

EPM.print_pseudotrial_stats(pseudotrials, trial_stats)

In [ ]:
pseudotrial_traces = {
    'open1': [],
    'open2': [],
    'closed1': [],
    'closed2': []
}

frames_per_pseudotrial = int(np.floor(PSEUDOTRIAL_LEN_S * ENDOSCOPE_FRAMERATE)) 

for arm in pseudotrials:
    arm_visits = pseudotrials[arm]
    
    for visit in arm_visits:
        start_index = visit['start']
        end_index = start_index + frames_per_pseudotrial
        traces = trimmed_trace_data[cell_names].iloc[start_index:end_index]
        # We first grab the columns for the cells and then grab the rows for our trial
        pseudotrial_traces[arm].append(traces)

In [ ]:
pseudotrial_means = {
    'open1': pd.DataFrame(),
    'open2': pd.DataFrame(),
    'closed1': pd.DataFrame(),
    'closed2': pd.DataFrame(),
}

for arm in pseudotrial_means:
    for trial in pseudotrial_traces[arm]:
        pseudotrial_means[arm] = pd.concat((pseudotrial_means[arm], trial.mean(axis=0)), axis=1)
    
    pseudotrial_means[arm] = pseudotrial_means[arm].T
    pseudotrial_means[arm] = pseudotrial_means[arm].reset_index(drop=True)

In [ ]:
folder = project_folder.analysis_dir.output_dir.subdir('pseudotrials')

IO.save_data_to_disk(pseudotrials, 'pseudotrials', file_header, folder)
IO.save_data_to_disk(trial_stats, 'trial_stats', file_header, folder)
IO.save_data_to_disk(transitions, 'transitions', file_header, folder)
IO.save_data_to_disk(arm_indexes, 'arm_indexes', file_header, folder)
IO.save_data_to_disk(pseudotrial_traces, 'pseudotrial_traces', file_header, folder)
IO.save_data_to_disk(pseudotrial_means, 'pseudotrial_means', file_header, folder)

In [ ]:
folder = project_folder.analysis_dir.output_dir.subdir('pseudotrials')

pseudotrials = IO.load_data_from_disk('pseudotrials', file_header, folder)
trial_stats = IO.load_data_from_disk('trial_stats', file_header, folder)
transitions = IO.load_data_from_disk('transitions', file_header, folder)
arm_indexes = IO.load_data_from_disk('arm_indexes', file_header, folder)
pseudotrial_traces = IO.load_data_from_disk('pseudotrial_traces', file_header, folder)
pseudotrial_means = IO.load_data_from_disk('pseudotrial_means', file_header, folder)

folder = project_folder.analysis_dir.preprocess_dir.path
curated_cell_props = IO.load_data_from_disk('curated_cell_props', file_header, folder)
cell_names = curated_cell_props['Name']

In [ ]:
from dewan_calcium import AUROC
groups = (['open1', 'open2'], ['closed1', 'closed2'])
# Open -> 0, closed -> 1
AUROC_results = AUROC.EPM_auroc(pseudotrial_means, groups, cell_names)

In [ ]:
folder = project_folder.analysis_dir.output_dir.subdir('AUROC')
IO.save_data_to_disk(AUROC_results, 'AUROC_results', file_header, folder)

In [ ]:
folder = project_folder.analysis_dir.output_dir.subdir('AUROC')
AUROC_results = IO.load_data_from_disk('AUROC_results', file_header, folder)

In [ ]:
#auroc_output = pd.DataFrame(columns=['auROC', 'direction_index', 'bounds', 'significant'])
auroc_output = []
for cell in AUROC_results:
    data = AUROC_results[cell]
    direction_index = round(2 * (data['auroc'] - 0.5), 2)
    auroc = round(data['auroc'], 2)
    bounds = (data['lb'], data['ub'])
    significance = data['significance']

    new_row = [auroc, direction_index, bounds, significance]
    auroc_output.append(new_row)
    
auroc_output = pd.DataFrame(auroc_output, index=cell_names, columns=['auROC', 'direction_index', 'bounds', 'significant'])

In [ ]:
# TODO: Add pseudotrial statistics to the excel output

In [ ]:
folder = project_folder.analysis_dir.output_dir.path
file_name = f'{file_header}EPM_data_output.xlsx'
file_path = folder.joinpath(file_name)
auroc_output.to_excel(file_path)

In [ ]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

auroc_vals = [AUROC_results[cell]['auroc'] for cell in AUROC_results]
auroc_vals = np.array(auroc_vals)
direction_indexes = 2 * (auroc_vals - 0.5)
fig, ax = plt.subplots(1,2, figsize=(8, 3))
ax[0].hist(direction_indexes, bins=20)
ax[1].hist(auroc_vals, color='r', bins=20)
minmax = [min(direction_indexes), max(direction_indexes)]
minmax2 = [min(auroc_vals), max(auroc_vals)]

ax[0].set_xticks(np.linspace(-1, 1, 21))
ax[1].set_xticks(np.linspace(0, 1, 21))

ax[0].set_xlim(minmax)
ax[1].set_xlim(minmax2)

ax[0].tick_params(axis='x', labelrotation=50)
ax[1].tick_params(axis='x', labelrotation=50)

ax[0].set_title('Direction Indices')
ax[1].set_title('auROC Values')
plt.tight_layout()

fig_dir = project_folder.analysis_dir.figures_dir.subdir('AUROC')
fig_path = fig_dir.joinpath('auROC_distribution.pdf')
fig.savefig(fig_path, dpi=600)
plt.close(fig)

for cell in tqdm(cell_names):
    try:
        results = AUROC_results[cell]
        ub = results['ub']
        lb = results['lb']
        auroc = results['auroc']
        shuffle = results['shuffle']
        significance = results['significance']
        
        relaxed_bounds = np.percentile(shuffle, [5, 95])
        lb_r, ub_r = relaxed_bounds
        
        fig, ax = plt.subplots()
        ax.hist(shuffle, color='gray', bins=10)
        ax.axvline(x=ub, color='blue')
        ax.axvline(x=lb, color='blue')
        ax.axvline(x=ub_r, color='green')
        ax.axvline(x=lb_r, color='green')
        ax.axvline(x=auroc, color='red')
        
        if significance in (-1, 1):
            ax.set_xlabel('Significant!')
        
        fig.suptitle(f'{cell} - {round(auroc, 3)}')
        
        save_dir = project_folder.analysis_dir.figures_dir.subdir('AUROC')
        file_path = save_dir.joinpath(f'{cell}.pdf')
        fig.savefig(file_path, dpi=600)
        plt.close(fig)    
    except Exception as e: # yes, this is bad; its okay
        print(f'Error plotting {cell}')
        print(e)
        

In [ ]:
# closed_distances = smoothed_traces[np.isin(smoothed_traces['Location'], ['closed1', 'closed2'])]['Distance'].values
# open_distances = smoothed_traces[np.isin(smoothed_traces['Location'], ['open1', 'open2'])]['Distance'].values * -1
# smoothed_traces.loc[np.isin(smoothed_traces['Location'], ['open1', 'open2']), "Distance"] *= -1
# smoothed_traces.loc[np.isin(smoothed_traces['Location'], ['center']), "Distance"] = 0
# from dewan_calcium import AUROC
# import matplotlib.pyplot as plt
# cell_avg_distances = {}
# for cell in tqdm(cell_names):
#
#     spikes = fake_spike_indexes[cell]
#     null_distances = []
#     num_spikes = len(spikes)
#
#     for _ in range(1000):
#         random_spike_indices = np.random.randint(0, len(smoothed_traces['Distance'].values), num_spikes)
#         random_distances = smoothed_traces['Distance'].values[random_spike_indices]
#         avg_distance = np.mean(random_distances)
#         null_distances.append(avg_distance)
#
#     null_distances = np.array(null_distances)
#     distance = smoothed_traces.iloc[spikes]['Distance']
#     cell_avg_distance = np.mean(distance)
#     ptile = np.percentile(null_distances, [1, 99])
#     cell_percentile = AUROC.compute_percentile(cell_avg_distance, null_distances)
#     # print(ptile, cell_percentile)
#
#     fig, ax = plt.subplots()
#
#     ax.hist(null_distances)
#     ax.vlines(x=cell_avg_distance, ymin=0, ymax=350, color='r')
#     ax.vlines(x=ptile, ymin=0, ymax=350, color='magenta')
#     fig.suptitle(f'{cell} - {cell_percentile*100}')
#     cell_avg_distances[cell] = cell_avg_distance
# coordinates = smoothed_traces['Coordinate']
# image_shape = image.shape
#
# combined_activity_heatmap, cell_heatmaps = EPM.generate_activity_heatmap(coordinates, fake_spike_indexes, cell_names, image_shape)
# line_coordinates = EPM.generate_position_lines(coordinates)
# folder = project_folder.analysis_dir.figures_dir.path.joinpath('heatmaps')
# folder.mkdir()
#
# filtered_spike_heatmap = ndimage.gaussian_filter(combined_activity_heatmap, sigma=1.5)
#
# for cell in cell_names:
#     heatmap = cell_heatmaps[cell]
#     filtered_heatmap = ndimage.gaussian_filter(heatmap, sigma=1.5)
#     plotting.plot_activity_heatmap(filtered_heatmap, line_coordinates, image, cell, folder)
#
#
# plotting.plot_activity_heatmap(filtered_spike_heatmap, line_coordinates, image, 'Combined', folder)  # Combined Heatmap